**# Importanto o Dataset e verificando o conteúdo**

In [1]:
import sys
import os
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords') #carrego stopwrds para filtragem NLP

# Adiciona o caminho da pasta onde o arquivo 'dataset.py' está localizado
# Isso pula a necessidade de mencionar a pasta com hífen no import
caminho_raw = os.path.abspath(os.path.join('..', 'data', 'raw'))
if caminho_raw not in sys.path:
    sys.path.append(caminho_raw)

from dataset import carregar_dados

df = carregar_dados()
print(df.head())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                           issue_url  \
0  "https://github.com/zhangyuanwei/node-images/i...   
1     "https://github.com/Microsoft/pxt/issues/2543"   
2  "https://github.com/MatisiekPL/Czekolada/issue...   
3  "https://github.com/MatisiekPL/Czekolada/issue...   
4  "https://github.com/MatisiekPL/Czekolada/issue...   

                                         issue_title  \
0  can't load the addon. issue to: https://github...   
1  hcl accessibility a11yblocking a11ymas mas4.2....   
2  issue 1265: issue 1264: issue 1261: issue 1260...   
3  issue 1266: issue 1263: issue 1262: issue 1259...   
4  issue 1288: issue 1285: issue 1284: issue 1281...   

                                                body  
0  can't load the addon. issue to: https://github...  
1  user experience: user who depends on screen re...  
2  ┆attachments: <a href= https:& x2f;& x2f;githu...  
3  gitlo = github x trello\n---\nthis board is no...  
4  ┆attachments: <a href= https:& x2f;& x2f;githu..

c:\Users\rafae\Desktop\Portifolio\AI_ASSISTANT_GITHUB\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rafae\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Processamento e limpeza dos dados**


In [2]:
sys.path.append(os.path.abspath(os.path.join('..', 'data', 'raw')))
sys.path.append(os.path.abspath(os.path.join('..', 'data', 'processed')))


from dataset import verifica_vazios, verifica_frequentes
from data_processing import clean_text

df["clean_title"] = df["issue_title"].apply(clean_text)
df["clean_body"] = df["body"].apply(clean_text)


media_palavras_url = df["issue_url"].str.split().str.len().mean()
media_palavras_title = df["clean_title"].str.split().str.len().mean()
media_palavras_body = df["clean_body"].str.split().str.len().mean()

print(f"Média de palavras em url: {media_palavras_url}, em title: {media_palavras_title}, em body: {media_palavras_body}")

url_vazio = verifica_vazios(df,"issue_url")
titles_vazio = verifica_vazios(df,"clean_title")
body_vazio = verifica_vazios(df,"clean_body")

palavras_frequentes_titles = verifica_frequentes(df,"clean_title", 20)
print(f"Palavras mais frequentes em titles: {palavras_frequentes_titles}")
palavras_frequentes_body = verifica_frequentes(df,"clean_body", 20)
print(f"Palavras mais frequentes em body: {palavras_frequentes_body}")



Média de palavras em url: 1.0, em title: 5.459, em body: 28.4455
Resultados para 'issue_url':
- Valores Nulos (NaN): 0
- Textos Vazios/Espaços: 0
Resultados para 'clean_title':
- Valores Nulos (NaN): 0
- Textos Vazios/Espaços: 99
Resultados para 'clean_body':
- Valores Nulos (NaN): 0
- Textos Vazios/Espaços: 11
Palavras mais frequentes em titles: [('issu', 352), ('add', 151), ('bug', 137), ('found', 79), ('error', 49), ('test', 43), ('use', 39), ('api', 39), ('help', 34), ('fix', 34), ('new', 33), ('need', 26), ('log', 24), ('run', 23), ('list', 22), ('get', 20), ('set', 19), ('line', 17), ('typo', 17), ('task', 16)]
Palavras mais frequentes em body: [('close', 1244), ('add', 1102), ('issues', 927), ('columns', 714), ('list', 585), ('comment', 558), ('column', 542), ('trello', 529), ('update', 396), ('default', 387), ('settings', 378), ('via', 376), ('sync', 373), ('move', 369), ('custom', 367), ('board', 358), ('card', 357), ('attachments', 355), ('cards', 353), ('matisiekpl', 352)]



   **Análise descritiva dos dados processados**

- Média de palavras para entender quantos tokens irão ser processados pela LLM
    - Quantidade de células vazias em todas as colunas
        - não teve resultado de células vazias, todas preenchidas
    - Títulos - tamanhos médio de 5,4 palavras
    - Body - Aproximadamente 28 palavras
- Ideal para:
    - Embeddings
    - Chunking Leve
    - RAG Eficiente(baixo custo e boa semântica)

**Criação de Texto final**

In [3]:
from sentence_transformers import SentenceTransformer

#criação de nova coluna para texto final que será direcionado ao embedding
df["final_text"] = (
    "Title: " + df["clean_title"] +
    ". Body: " + df["clean_body"]
)

#carregando o modelo que será usado, modelo rápido e leve para projeto OBS: Uso da CPU pois GPU esta ultrapassada para o modelo
model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2", device="cpu") #usando o paraphrasal para poder perguntar em portugues
df[["clean_title", "clean_body", "final_text"]].head()

Loading weights: 100%|██████████| 199/199 [00:00<00:00, 850.83it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


,clean_title,clean_body,final_text
0,load addon issue error lib libc version glibc ...,load addon issue error lib libc version glibc ...,Title: load addon issue error lib libc version...
1,hcl accessibility yblocking ymas mas hcl makec...,user experience user depends screen reader get...,Title: hcl accessibility yblocking ymas mas hc...
2,issue issue issue issue issue issue issue issu...,attachments github com matisiekpl czekolada is...,Title: issue issue issue issue issue issue iss...
3,issue issue issue issue issue issue issue issu...,gitlo github trello board linked update issue ...,Title: issue issue issue issue issue issue iss...
4,issue issue issue issue issue issue issue issu...,attachments github com matisiekpl czekolada is...,Title: issue issue issue issue issue issue iss...


**Embedding**

In [4]:
import numpy as np

#listando o texto final em variavel para ser codificada
texts = df["final_text"].tolist()

#realização do embedding pelo modelo escolhido
embeddings = model.encode( 
    texts,
    show_progress_bar= True
)

#criação do array em np 
embeddings = np.array(embeddings)
print(embeddings.shape)

#persistindo os dados em formato npy e csv para nao necessitar de conversao novamente
np.save("embeddings.npy",embeddings)
df.to_csv("issue_processed.csv", index=False)


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

Batches: 100%|██████████| 63/63 [00:15<00:00,  4.02it/s]

(2000, 384)


**Busca Semântica**

- Primeiramente utilizarei busca ingênua para este caso, mais rápido e ideal para projetos pequenos.
    - será utilizado semelhança de cossenos, variância de -1 a 1, sendo 1 o mais próximo.
    - comparação de vetores do embedding, modelo utilizado de dimensão 384
- Após implementado e projeto funconando, irei dar updgrade para utilizar índice vetorial
- Devido a arquitetura proposta relacionado aos tipos de perguntas para o Chatbot, será implementado:
    - Busca semântica
    - Top k
    - Limiar de similiaridade
    Dessa forma será possível a resposta de perguntas analtícas e explicativas, não somente localizadoras.

In [5]:
import pandas as pd #ler diretamente esta célula sem precisar converter novamente os arquivos no embedding
import numpy as np

embeddings = np.load("embeddings.npy")
df = pd.read_csv("issue_processed.csv")

**Exemplo de pergunta e formato para query**

In [22]:
from semantic_search import encode_query, cosine_similiarity_func

pergunta = "Quais são os erros mais recorrentes no documento?"
query = encode_query(model, [pergunta])
#print(query.shape)

scores = cosine_similiarity_func(query,embeddings)
scores = scores.flatten()
scores_ordenados = np.argsort(scores)
print(scores.shape)
print(scores_ordenados)

#fazendo busca top_k, depois refatorar montando em funcoes definidas
# Atualizar o Readme e usar o topk como parametro para busca ao refatorar “Utilizamos top-k dinâmico para balancear cobertura semântica e precisão.”
top_k = 30
top_indices = scores_ordenados[-top_k:][::-1] #quero buscar os ultimos 20 valores e ordena-los em sequência maior para menor, pois np.arg retorna ordem crescente
print(top_indices)




(2000,)
[ 914 1965  889 ...  885 1725  571]
[ 571 1725  885 1729  976  736  844 1736 1637  549  539  596  494  959
 1082  538  583 1635 1513 1872 1342 1625 1659  522  462  698  419 1476
  356 1630]


**Implementando Limiar de similaridade**

 - Utilizei um limiar de similaridade cosseno ajustado empiricamente para garantir que apenas documentos semanticamente relevantes sejam utilizados como contexto para a LLM, reduzindo ruído e alucinação.
 - A princípio foi definido padrão = 0.30, após verifiquei de forma empirica os melhores valores para retornar respostas completas sem interferências de dados fora do padrão

In [23]:
#definir o limiar, fazer um loop retornando os melhores índices
limiar = 0.30
lista_final = []

for i in range(len(top_indices)):
    if scores[top_indices[i]] > limiar:
        lista_final.append({"indice" : top_indices[i] ,"score" :scores[top_indices[i]], "text" : df.iloc[top_indices[i]]["final_text"]})

print(lista_final)

[{'indice': np.int64(571), 'score': np.float32(0.42988878), 'text': 'Title: . Body: according demo code returns standard errors according code column naming seems return standard deviations meantime assuming model returns standard deviations using boot boot empirical standard errors slow already including less unwieldy method computing kind standard errors report paper would happy clarification also information getting code work kept running error error abs max beta beta tol maxit missing value true false needed ultimately got around explicitly setting intercept input formula model matrix computed model matrix zero'}, {'indice': np.int64(1725), 'score': np.float32(0.41551167), 'text': 'Title: err code. Body: believe past issue regarding error message get run test dataset two output files params txt spades log help would greatly appreciated thanks marybel'}, {'indice': np.int64(885), 'score': np.float32(0.41383582), 'text': 'Title: typo. Body: basic security testing ios word repeated tw

**Construção do contexto**

- Utilizar dos dados retornados a partir do limiar e construir os textos mais similares para servir de entrada para LLM
- estou retornando em texto todos os documentos analisados que passaram pelo limiar, poderia restringir a quantidade caso o número de tokens a ser utilizado na LLM seja limitado

In [24]:
MAX_CHARS = 800  # por documento

lista_final = sorted(
    lista_final,
    key=lambda x: x["score"],
    reverse=True
)

contexto = ""
limite = 28

if len(lista_final) <= limite:
    for i in range(len(lista_final)):
        text_limited = lista_final[i]["text"][:MAX_CHARS]

        contexto += (
            f"[Contexto {i+1} | "
            f"Similaridade: {lista_final[i]['score']:.03f}]\n"
            f"{text_limited}\n\n"
        )

else:
    for i in range(limite):
        text_limited = lista_final[i]["text"][:MAX_CHARS]

        contexto += (
            f"[Contexto {i+1} | "
            f"Similaridade: {lista_final[i]['score']:.03f}]\n"
            f"{text_limited}\n\n"
        )

print(contexto)

[Contexto 1 | Similaridade: 0.430]
Title: . Body: according demo code returns standard errors according code column naming seems return standard deviations meantime assuming model returns standard deviations using boot boot empirical standard errors slow already including less unwieldy method computing kind standard errors report paper would happy clarification also information getting code work kept running error error abs max beta beta tol maxit missing value true false needed ultimately got around explicitly setting intercept input formula model matrix computed model matrix zero

[Contexto 2 | Similaridade: 0.416]
Title: err code. Body: believe past issue regarding error message get run test dataset two output files params txt spades log help would greatly appreciated thanks marybel

[Contexto 3 | Similaridade: 0.414]
Title: typo. Body: basic security testing ios word repeated twice blatant tampering course invalidates code signature

[Contexto 4 | Similaridade: 0.412]
Title: error 

**Integrando LLM**

- Estou usando Gemini devido a facilidade de obtenção de uma chave gratuita para estudantes
- A LLM irá receber as informações retiradas do Dataset 
- Será utilizado engenharia de prompt para qualificar a análise da LLM
- No projeto será implantado um scan de input para o usuário digitar sua propria chave api gemini e testar o programa

In [25]:
sys.path.append(os.path.abspath(os.path.join('..', 'src', 'prompts')))
sys.path.append(os.path.abspath(os.path.join('..', 'src', 'llm')))

from dotenv import load_dotenv
import os
from rag_prompts import build_prompts
from gemini_client import generate_answer

load_dotenv()
chave_api = os.getenv("GOOGLE_API_KEY")

prompt = build_prompts(contexto, pergunta)
resposta = generate_answer(prompt)

print(resposta)


Com base nos contextos fornecidos, os erros mais recorrentes parecem estar relacionados a:

*   **Erros de execução/travamentos (crashes):** Vários títulos e corpos de texto mencionam "crash", "crashes impacted devices", "brf number crashes" e "crashlytics".
*   **Problemas com testes e bugs:** Termos como "bug", "test", "errors", "issue", "fix" e "found bug" aparecem com frequência, indicando dificuldades em testes e a descoberta de erros.
*   **Erros de processamento de dados/resultados:** Contextos como "standard errors", "standard deviations", "missing value", "incorrect info", "null" e "data inserted" sugerem problemas na forma como os dados são processados, calculados ou inseridos.
*   **Erros de código ou sintaxe:** Menciona-se "error code", "err code", "typo", "invalidates code signature", "error compilechunk" e "unboundlocalerror", indicando falhas na escrita ou interpretação do código.
